Some code to create (train, val, test) set from dataset composed of multiple data `.pkl` files.

This is no longer actively being used to create validation sets. Use a python script instead.

In [7]:
import os
import shutil
import dill
import random
from glob import glob
import numpy as np
from tqdm.notebook import tqdm
from collect.generate.scene.v3.trajectron_scene import augment_scene

DATADIR = 'rawCarla-0.9.13-Town03'
[os.path.abspath(path) for path in glob(f"{ DATADIR }/*.pkl")]

['/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split2_train.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split1_train.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split3_val.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split2_val.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split3_test.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split1_train_modfm.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split1_val.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split2_test.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split1_test.pkl',
 '/home/daedalus/code/robotics/carla-collect/carla_dataset_v4-2/v4-2_split3_train.pkl']

Copy the output array into the next Jupyter notebook cell

In [8]:
ds = ['/home/daedalus/code/robotics/carla-collect/rawCarla-0.9.13-Town03/20220326_20-29-56_Town03.pkl',
 '/home/daedalus/code/robotics/carla-collect/rawCarla-0.9.13-Town03/20220326_19-45-40_Town03.pkl',
 '/home/daedalus/code/robotics/carla-collect/rawCarla-0.9.13-Town03/20220326_20-18-10_Town03.pkl',
 '/home/daedalus/code/robotics/carla-collect/rawCarla-0.9.13-Town03/20220326_20-35-50_Town03.pkl',
 '/home/daedalus/code/robotics/carla-collect/rawCarla-0.9.13-Town03/20220326_20-24-03_Town03.pkl']
len(ds)

1

## Inspect the dataset

In [9]:
with open(ds[0], 'rb') as f:
    env = dill.load(f, encoding='latin1')
len(env.scenes)

800

In [4]:
env.scenes[0].name

'Carla/Maps/Town03/ep015/agent1679/frame00002176'

## Creating (train, val, test) sets from dataset.

In [7]:
n_pkl_to_train_set = 1
val_set_idx  = 3
test_set_idx = 4
val_set_size = 30
test_set_size = 30
version_label = "v4_1"

In [21]:
# train set
print("Forming train set")
with open(ds[0], 'rb') as f:
    env = dill.load(f, encoding='latin1')
for dataset in ds[1:n_pkl_to_train_set]:
    with open(dataset, 'rb') as f:
        env2 = dill.load(f, encoding='latin1')
    env.scenes.extend(env2.scenes)
print(f"Got {len(env.scenes)} scenes")

print("Augmenting scenes")
for scene in tqdm(env.scenes):
    scene.augmented = list()
    angles = np.arange(0, 360, 15)
    for angle in angles:
        scene.augmented.append(augment_scene(scene, angle))

print("Shuffling scenes")
random.shuffle(env.scenes)
random.shuffle(env.scenes)
        
print("Saving train set")
savepath = f"{ DATADIR }/carla_train_{ version_label }_full.pkl"
with open(savepath, 'wb') as f:
    dill.dump(env, f, protocol=dill.HIGHEST_PROTOCOL)

print("Saving val set")
savepath = f"{ DATADIR }/carla_val_{ version_label }_full.pkl"
if val_set_size is None:
    shutil.copyfile(ds[val_set_idx], os.path.abspath(savepath))
else:
    with open(ds[val_set_idx], 'rb') as f:
        env = dill.load(f, encoding='latin1')
    print(f"Has {len(env.scenes)} scenes, selecting {val_set_size} of them")
    random.shuffle(env.scenes)
    env.scenes = env.scenes[:val_set_size]
    with open(savepath, 'wb') as f:
        dill.dump(env, f, protocol=dill.HIGHEST_PROTOCOL)

print("Saving test set")
savepath = f"{ DATADIR }/carla_test_{ version_label }_full.pkl"
if test_set_size is None:
    shutil.copyfile(ds[test_set_idx], os.path.abspath(savepath))
else:
    with open(ds[test_set_idx], 'rb') as f:
        env = dill.load(f, encoding='latin1')
    print(f"Has {len(env.scenes)} scenes, selecting {test_set_size} of them")
    random.shuffle(env.scenes)
    env.scenes = env.scenes[:test_set_size]
    with open(savepath, 'wb') as f:
        dill.dump(env, f, protocol=dill.HIGHEST_PROTOCOL)

print("Done")

Forming train set
Got 209 scenes
Augmenting scenes



Shuffling scenes
Saving train set
Saving val set
Has 203 scenes, selecting 30 of them
Saving test set
Has 211 scenes, selecting 30 of them
Done


### Hotfix 1

In [40]:
HOTFIX_PREFIX = "Carla/Maps/"
def hotfix_name(name):
    """In CARLA 0.9.13 generated raw data,
    names may accidently have the form 'Carla/Maps/TownXX/...'.
    Return name with the prefix 'Carla/Maps' removed."""
    if name.startswith(HOTFIX_PREFIX):
        return name[len(HOTFIX_PREFIX):]
    return name

datafile = ds[4]

In [41]:
with open(datafile, 'rb') as f:
    env = dill.load(f, encoding='latin1')
print(datafile)
print(env.name)
print(env.scenes[0].name)
print(env.scenes[1].name)
print(env.scenes[-2].name)
print(env.scenes[-1].name)

/home/daedalus/code/robotics/carla-collect/rawCarla-0.9.13-Town03/20220326_20-24-03_Town03.pkl
Town03
Town03/ep010/agent894/frame00002266
Town03/ep010/agent895/frame00002266
Town03/ep014/agent1472/frame00004844
Town03/ep014/agent1473/frame00004844


In [28]:
env.name = hotfix_name(env.name)
for scene in env.scenes:
    scene.name = hotfix_name(scene.name)

In [29]:
print(datafile)
print(env.name)
print(env.scenes[0].name)
print(env.scenes[1].name)
print(env.scenes[-2].name)
print(env.scenes[-1].name)

/home/daedalus/code/robotics/carla-collect/rawCarla-0.9.13-Town03/20220326_20-24-03_Town03.pkl
Town03
Town03/ep010/agent894/frame00002266
Town03/ep010/agent895/frame00002266
Town03/ep014/agent1472/frame00004844
Town03/ep014/agent1473/frame00004844


In [30]:
with open(datafile, 'wb') as f:
    dill.dump(env, f, protocol=dill.HIGHEST_PROTOCOL)